# Preparing workspace

In [ ]:
#r "netstandard"
#r @"bin\Cntk.Core.Managed-2.6.dll"
#load @".paket\load\main.group.fsx"
 
open System
open System.IO

Environment.GetEnvironmentVariable("PATH")
|> fun path -> sprintf "%s%c%s" path (Path.PathSeparator) (Path.GetFullPath("bin"))
|> fun path -> Environment.SetEnvironmentVariable("PATH", path)

open CNTK
DeviceDescriptor.UseDefaultDevice().Type
|> printfn "Congratulations, you are using CNTK for: %A" 

In [ ]:
let device = CNTK.DeviceDescriptor.CPUDevice
let dataType = CNTK.DataType.Float
let initialization = CNTKLib.GlorotUniformInitializer(1.0)
let input_dim, num_output_classes = 2,2

#load "fsx/CntkHelpers.fsx"
open CntkHelpers

let input = Variable.InputVariable(shape [|input_dim|], dataType, "Features")
let label = Variable.InputVariable(shape [|num_output_classes|], dataType, "Labels")
let z = fullyConnectedClassifierNet input [15;10;5] num_output_classes CNTKLib.Sigmoid

In [ ]:
open System.Collections.Generic

// combine filters to single lambda
let inline (<&>) (filter1 : 'T -> bool) (filter2 : 'T -> bool) =
        fun (t:'T) -> filter1 t && filter2 t

let decomposeFunction (root: Function) = 
        
        let visited = Dictionary<string, Variable>()
        
        let inline excludeVisitedNodes (v: Variable) = 
            if visited.ContainsKey(v.Uid) then false
            else visited.Add(v.Uid, v); true
    
        let inline includeNodesWithOwner (v: Variable) = v.Owner |> isNull |> not             

        let rec search (f: Function) = 
        
            visited.Add(f.Uid, Var f)
        
            seq {   yield! f.Inputs
                    yield! f.Outputs
                    yield  Var f.RootFunction }
            |> Seq.filter (excludeVisitedNodes <&> includeNodesWithOwner)
            |> Seq.map (fun v -> v.Owner)
            |> Seq.filter ((<>)null)
            |> Seq.iter search

        search root
    
        visited

In [ ]:
decomposeFunction z

In [ ]:
// https://github.com/Microsoft/CNTK/blob/master/bindings/python/cntk/logging/graph.py

let extractGraphVizDotNotation (f: Function) = 
        let varText (v:Variable) = (if String.IsNullOrEmpty v.Name then v.Uid else v.Name) + "\\n" + v.Shape.AsString()
        let funText (f: Function) = if String.IsNullOrEmpty f.Name then f.Uid else f.Name
    
        let varLabel (v: Variable) = sprintf "%s [label=\"%s\"];" v.Uid (varText v)
        let funLabel (f: Function) = sprintf "%s [label=\"%s\"];" f.Uid (funText f)

        let varShape (v: Variable) =
            match v with
            | _ when v.IsInput -> sprintf "%s [shape=invhouse, color=yellow];" v.Uid
            | _ when v.IsOutput -> sprintf "%s [shape=invhouse, color=gray];" v.Uid
            | _ when v.IsPlaceholder -> sprintf "%s [shape=invhouse, color=yellow];" v.Uid
            | _ when v.IsParameter -> sprintf "%s [shape=diamond, color=green];" v.Uid
            | _ when v.IsConstant -> sprintf "%s [shape=rectangle, color=lightblue];" v.Uid
            | _ -> sprintf "%s [shape=circle, color=purple];" v.Uid

        let funShape (f: Function) = 
            match f with 
            | _ when f.IsComposite -> sprintf "%s [shape=ellipse, fontsize=20, penwidth=2, peripheries=2];" f.Uid
            | _ when f.IsPrimitive -> sprintf "%s [shape=ellipse, fontsize=20, penwidth=2, size=0.6];" f.Uid
            | _ -> sprintf "%s [shape=ellipse, fontsize=20, penwidth=4];" f.Uid

        let varEdges (f: Function) (v: Variable) = 
            let inputIndex = f.Inputs |> Seq.map (fun v -> v.Uid) |> Set
            let outputIndex = f.Outputs |> Seq.map (fun v -> v.Uid) |> Set
            match inputIndex.Contains(v.Uid), outputIndex.Contains(v.Uid) with 
            | true, _ when v.IsParameter -> sprintf "%s -> %s [label=\"input param\"];" v.Uid f.Uid |> Some
            | _, true when v.IsParameter -> sprintf "%s -> %s [label=\"output param\"];" f.Uid v.Uid|> Some
            | true, _ -> sprintf "%s -> %s [label=input];" v.Uid f.Uid|> Some
            | _, true -> sprintf "%s -> %s [label=output];" f.Uid v.Uid|> Some
            //| _ when v.IsParameter -> sprintf "%s -> %s [label=param];" f.Uid v.Uid|> Some
            | _ -> None //sprintf "%s -> %s;" f.Uid v.Uid

        let vars = Seq.append f.Inputs f.Outputs
        let funs = seq { 
                yield f
                yield f.RootFunction;
                yield! vars |> Seq.filter (fun v -> v.Owner |> isNull |> not) |> Seq.map (fun v -> v.Owner) 
            } 

        seq {        
            if f.Uid <> f.RootFunction.Uid then yield sprintf "%s -> %s [label=\"root function\"];" f.RootFunction.Uid f.Uid
            yield! vars |> Seq.map varShape
            yield! vars |> Seq.map varLabel
            yield! vars |> Seq.map (varEdges f) |> Seq.choose id
            yield! funs |> Seq.map funLabel 
            yield! funs |> Seq.map funShape
        } |> Seq.distinct


In [ ]:
let createGraphVizDiagram (f:Function) =
    f 
    |> decomposeFunction 
    |> Seq.cast<KeyValuePair<string,Variable>> 
    |> Seq.map(fun pair -> pair.Value) 
    //|> Seq.where(fun v -> v.Owner)
//     |> Seq.where (function Fun _ -> true | _ -> false)
//    |> Seq.where (fun v -> v.ToFunction().IsComposite |> not )
    |> Seq.collect (fun v -> extractGraphVizDotNotation (v.ToFunction()))
    |> Seq.distinct //|> Array.ofSeq |> Array.filter (fun str -> str.Contains("->")) |> Array.sort
    |> Seq.sort
    
    
createGraphVizDiagram z

In [ ]:
let dotNotationString = createGraphVizDiagram z |> Seq.reduce(sprintf "%s\n%s")
let replace pattern (replacement: string) (str: string) = str.Replace(pattern,replacement)

"d3-graphviz-host.html" 
|> System.IO.File.ReadAllText 
|> replace "{DOT}" dotNotationString
|> Util.Html
|> Display

In [ ]:
//(Var z).GetType().GetProperties() |> Array.map (fun prop -> prop.Name)
let v = z
v.GetType().GetProperties() 
|> Array.map (
    fun prop ->
        try        
            prop.Name, (sprintf "%A" <| prop.GetValue(v))
        with ex -> prop.Name, "")
|> Util.Table        

In [ ]:
let v = input
v.GetType().GetProperties() 
|> Array.map (
    fun prop ->
        try        
            prop.Name, (sprintf "%A" <| prop.GetValue(input))
        with ex -> prop.Name, "")
|> Util.Table        

In [ ]:
let z' = Var z
z'.Owner.Name

In [ ]:
@"<script src=""https://d3js.org/d3.v5.min.js""></script>" |> Util.Html |> Display

In [ ]:
@"<script src='http://d3js.org/d3.v5.min.js'></script>" |> Util.Html

In [ ]:
@"<script src='https://unpkg.com/viz.js@1.8.0/viz.js' type='javascript/worker'></script>" |> Util.Html

In [ ]:
@"<script src='https://unpkg.com/d3-graphviz@1.3.1/build/d3-graphviz.min.js'></script>" |> Util.Html